# Imports

In [1]:
%run Imports.ipynb
name = 'Kred' # Choose Kred or Mone

param_grid = {
    'learning_rate': [0.001, 0.005, 0.01, 0.05],
    'max_leaves': [3, 5, 10, 15, 20, 32],
    'min_samples_leaf': [2, 5, 10, 15, 20],
    'interactions': [5, 10, 15]
}

/home/RDC/fliegebe/.conda/envs/Work/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# 1) Read files

In [2]:
key_featsubgroups = pd.read_pickle('../../pickle/2_FS/' + name + '/key_featsubgroups.pkl')
df = pd.read_pickle('../../pickle/2_FS/' + name + '/2_df_new_.pkl')

In [3]:
def split_data_4(df, key_featsubgroups=key_featsubgroups, target=target, test_size=0.2, random_state=42):
    """
    Splits the dataset into training and testing sets while excluding demographic features.

    Parameters:
    df (DataFrame): The dataset containing features and target variable.
    key_featsubgroups (DataFrame): A mapping of feature subgroups.
    target (str): The name of the target variable.
    test_size (float, optional): The proportion of the dataset to allocate for testing. Default is 0.2.
    random_state (int, optional): Random seed for reproducibility. Default is 42.

    Returns:
    tuple: X_train, X_test, y_train, y_test (training and testing datasets)
    """

    # Extract demographic features
    demo_feat = key_featsubgroups.loc[key_featsubgroups['subgroup'] == 'demo', 'list_features'].values[0]
    print("Demographic Features:", demo_feat)

    # Separate features (X) and target variable (y), excluding demographic features
    X = df.drop(columns=[target] + demo_feat)
    y = df[target]

    # Split the dataset into training (80%) and testing (20%) sets
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, random_state=random_state
    )

    # Print dataset shapes
    print(f"Training Features Shape: {X_train.shape}")
    print(f"Training Labels Shape: {y_train.shape}")
    print(f"Testing Features Shape: {X_test.shape}")
    print(f"Testing Labels Shape: {y_test.shape}")

    return X_train, X_test, y_train, y_test

In [11]:
# with open('../pickle/3_Model/xgb_default_params.pkl', 'rb') as f:
#     default_params = pickle.load(f)

# Load results_dict_updated from the pickle file
with open('../../pickle/4_Model_Optuna/results_dict_updated_5.pkl', 'rb') as f:
    results_dict = pickle.load(f)

In [12]:
results_dict.keys()

dict_keys(['rf0', 'xgbc0', 'xgbc_optuna', 'xgbc_optimized', 'ebm_default', 'ebm_ben', 'sklearn_nn'])

# 5.3) Optuna

In [13]:
def train_optuna_EBM(param_grid, df, results_dict):
    """
    Tunes and trains an Explainable Boosting Machine (EBM) via Optuna,
    evaluates performance, and stores results in the given dictionary.

    Parameters:
    -----------
    param_grid : dict
        Search space for Optuna. Keys are EBM init parameter names,
        values are lists of candidate values.
    df : DataFrame
        Dataset containing features and the target variable.
    results_dict : dict
        Dictionary to store model performance metrics.

    Returns:
    --------
    best_params : dict
        Best parameters found by Optuna.
    results_dict : dict
        Updated results dictionary containing model evaluation metrics.
    global_importance : object
        Global feature importance from the tuned EBM model.
    local_importance : object
        Local feature importance from the tuned EBM model.
    """
    start_time = time.time()
    print("\nStarting Optuna hyperparameter search for EBM...")

    # split_data_4 must return X_train, X_test, y_train, y_test
    X_train, X_test, y_train, y_test = split_data_4(df)

    # determine total number of trials
    n_trials = int(np.sum([len(v) for v in param_grid.values()]))
    print(f"No. of trials scheduled: {n_trials}")

    def objective(trial):
        # sample hyperparameters from the grid
        params = {
            key: trial.suggest_categorical(key, values)
            for key, values in param_grid.items()
        }
        # instantiate an EBM with these parameters
        ebm = ExplainableBoostingClassifier(**params)
        # cross‐validated accuracy on training set
        cv_scores = cross_val_score(
            ebm, X_train, y_train,
            cv=5, scoring="accuracy"
        )
        return cv_scores.mean()

    # run Optuna study
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=n_trials)

    best_params = study.best_params
    print("Best hyperparameters found:", best_params)

    # train final EBM with best params
    ebm_best = ExplainableBoostingClassifier(**best_params)
    ebm_best.fit(X_train, y_train)

    # cross‐validated predictions on train set
    y_train_pred = cross_val_predict(ebm_best, X_train, y_train, cv=5)
    # predictions on test set
    y_test_pred  = ebm_best.predict(X_test)
    y_test_proba = ebm_best.predict_proba(X_test)[:, 1]

    # compute metrics
    cfm_train    = confusion_matrix(y_train, y_train_pred)
    cfm_test     = confusion_matrix(y_test,  y_test_pred)
    acc_train    = accuracy_score(y_train, y_train_pred)
    acc_test     = accuracy_score(y_test,  y_test_pred)
    f1_train_1   = f1_score(y_train, y_train_pred, pos_label=1)
    f1_train_0   = f1_score(y_train, y_train_pred, pos_label=0)
    f1_test_1    = f1_score(y_test,  y_test_pred,  pos_label=1)
    f1_test_0    = f1_score(y_test,  y_test_pred,  pos_label=0)
    roc_auc_test = roc_auc_score(y_test, y_test_proba)

    # extract importances
    global_importance = ebm_best.explain_global().data()
    local_importance  = ebm_best.explain_local(X_test, y_test).data()

    total_time = (time.time() - start_time) / 60
    print(f"Optimized EBM training completed in {total_time:.2f} minutes")

    # store results
    results_dict["ebm_optuna"] = {
        "classifier":         deepcopy(ebm_best),
        "cfm_train":          cfm_train,
        "cfm_test":           cfm_test,
        "train_accuracy":     acc_train,
        "test_accuracy":      acc_test,
        "train F1-score label 1": f1_train_1,
        "train F1-score label 0": f1_train_0,
        "test F1-score label 1":  f1_test_1,
        "test F1-score label 0":  f1_test_0,
        "test roc auc score":     roc_auc_test,
        "default_params":         ebm_best.get_params(),
        "param_grid":             param_grid,
        "best_params":            best_params,
        "global_importance":      global_importance,
        "local_importance":       local_importance,
        "time_m":                 total_time
    }

    return best_params, results_dict, global_importance, local_importance


In [ ]:
best_params, results_dict, global_importance, local_importance = train_optuna_EBM(param_grid, df, results_dict)


Starting Optuna hyperparameter search for EBM...
Demographic Features: ['clientdata.demo.gender', 'clientdata.demo.age_year', 'clientdata.demo.age_month', 'clientdata.demo.children', 'clientdata.demo.children_singleparent', 'clientdata.demo.maritalstatus_expand_SINGLE', 'clientdata.demo.maritalstatus_expand_MARRIED', 'clientdata.demo.maritalstatus_expand_DIVORCED', 'clientdata.demo.maritalstatus_expand_WIDOWED', 'clientdata.demo.maritalstatus_expand_newvalue', 'clientdata.demo.maritalstatus_woe']


[I 2025-07-10 13:36:23,915] A new study created in memory with name: no-name-9b670e5a-c952-4e95-9f66-e0cd6ad00628


Training Features Shape: (103565, 406)
Training Labels Shape: (103565,)
Testing Features Shape: (25892, 406)
Testing Labels Shape: (25892,)
No. of trials scheduled: 18


In [ ]:
with open('../../pickle/4_Model_Optuna/results_dict_updated_5.pkl', 'wb') as f:
    pickle.dump(results_dict, f, pickle.HIGHEST_PROTOCOL)